In [1]:
import os
import sys
import gc
import pickle
import warnings
import pandas as pd
import numpy as np
import seaborn as sns
from typing import *
from tqdm.notebook import tqdm
from pathlib import Path
from matplotlib import pyplot as plt

pd.set_option('max_columns', 50)
pd.set_option('max_rows', 200)
warnings.simplefilter('ignore')
sns.set()

In [2]:
base_dir = Path().resolve()
sys.path.append(str(base_dir / '../'))

from utils.preprocess import *
from utils.model import *
from utils.train import *
from utils.eval import *
from utils.detectron2helper import *

fail to import apex_C: apex was not installed or installed without --cpp_ext.
fail to import amp_C: apex was not installed or installed without --cpp_ext.
** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 

** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 



HACKING: overriding COCOeval.summarize = vin_summarize...


In [3]:
def seed_everything(seed: int, device: str):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    with torch.cuda.device(device):
        torch.cuda.empty_cache()

In [4]:
from dataclasses import dataclass, field, asdict
import yaml


@dataclass
class Config:
    # General
    debug: bool = False
    outdir: str = "detectron2_results/[tmp]"
    device: str = "cuda:0"
    device_id: int = 0
    num_workers: int = 8

    # Data config
    imgconf_file: str = '../../data/VinBigData/train.csv'
    meta_file: str = '../../data/VinBigData/train_meta.csv'
    test_meta_file: str = '../../data/VinBigData/test_meta.csv'
    imgdir_name: str = "../../data/VinBigData/png[tmp]"
    seed: int = 111
    n_splits: int = 5
    iou_thr: float = 0.4
    skip_box_thr: float = 0.1
    sigma: float = 0.1
    
#     model_config: str = 'COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml'
#     model_config: str = 'COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml'
#     model_config: str = 'Misc/cascade_mask_rcnn_R_50_FPN_3x.yaml'
#     model_config: str = 'COCO-Detection/retinanet_R_101_FPN_3x.yaml'
#     model_config: str = 'COCO-Detection/retinanet_R_50_FPN_3x.yaml'
    
    # Training config
    batch_size: int = 8
    iter: int = 10000
    lr_scheduler_name: str = "WarmupCosineLR"  # WarmupMultiStepLR (default) or WarmupCosineLR
    base_lr: float = 0.00025
    roi_batch_size_per_image: int = 512
    eval_period: int = 10000
    checkpoint_period: int = 10000
        
    aug_kwargs: Dict = field(default_factory=lambda: {})
        

    def update(self, param_dict: Dict) -> "Config":
        # Overwrite by `param_dict`
        for key, value in param_dict.items():
            if not hasattr(self, key):
                raise ValueError(f"[ERROR] Unexpected key for flag = {key}")
            setattr(self, key, value)
        return self
    
    def to_yaml(self, filepath: str, width: int = 120):
        with open(filepath, 'w') as f:
            yaml.dump(asdict(self), f, width=width)

In [5]:
config_dict = {
    'debug': False,
    'batch_size': 25,
    'base_lr': 1e-3,
    'iter': 10000,
    'eval_period': 1000,
    'checkpoint_period': 1000,
    "aug_kwargs": {
        "HorizontalFlip": {"p": 0.5},
        "ShiftScaleRotate": {"scale_limit": 0.15, "rotate_limit": 10, "p": 0.5},
        "RandomBrightnessContrast": {"p": 0.5},
    },
    'iou_thr': 0.4,
    'skip_box_thr': 0.05,
}
config = Config().update(config_dict)

In [6]:
classes_nms = [
    "Aortic enlargement",
    "Atelectasis",
    "Calcification",
    "Cardiomegaly",
    "Consolidation",
    "ILD",
    "Infiltration",
    "Lung Opacity",
    "Nodule/Mass",
    "Other lesion",
    "Pleural effusion",
    "Pleural thickening",
    "Pneumothorax",
    "Pulmonary fibrosis",
#     "No Finding"
]
classes_dict = {index: class_name  for index, class_name in enumerate(classes_nms)}

In [7]:
test_meta = pd.read_csv(str(base_dir / config.test_meta_file))
# dataset_dicts_test = get_vinbigdata_dicts_test(
#     base_dir / config.imgdir_name, 
#     test_meta,
#     test_data_type=f'png{config.img_size}',
#     debug=config.debug
# )

In [8]:
img_size = 512
imgdir_name = config.imgdir_name.replace('[tmp]', str(img_size))

DatasetCatalog.register(
    f"vinbigdata_test_{img_size}", lambda: get_vinbigdata_dicts_test(imgdir_name, test_meta, debug=config.debug)
)
MetadataCatalog.get(f"vinbigdata_test_{img_size}").set(thing_classes=classes_nms)
metadata_512= MetadataCatalog.get(f"vinbigdata_test_{img_size}")
dataset_dicts_512 = get_vinbigdata_dicts_test(
    base_dir / imgdir_name, 
    test_meta,
    test_data_type=f'png{img_size}',
    debug=config.debug
)

img_size = 1024
imgdir_name = config.imgdir_name.replace('[tmp]', str(img_size))

DatasetCatalog.register(
    f"vinbigdata_test_{img_size}", lambda: get_vinbigdata_dicts_test(imgdir_name, test_meta, debug=config.debug)
)
MetadataCatalog.get(f"vinbigdata_test_{img_size}").set(thing_classes=classes_nms)
metadata_1024 = MetadataCatalog.get(f"vinbigdata_test_{img_size}")
dataset_dicts_1024 = get_vinbigdata_dicts_test(
    base_dir / imgdir_name, 
    test_meta,
    test_data_type=f'png{img_size}',
    debug=config.debug
)

Load from cache dataset_dicts_cache_test_png512_debug0.pkl
Load from cache dataset_dicts_cache_test_png1024_debug0.pkl


In [9]:
models = [
    ('faster_rcnn_R_101_FPN_3x_mkf_5_1024_rm', 'COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml', 1024, True),
    ('faster_rcnn_R_50_FPN_3x_mkf_5_1024', 'COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml', 1024, True),
    ('faster_rcnn_R_50_FPN_3x_mkf_5_1024_hmean', 'COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml', 1024, True),
    ('retinanet_R_101_FPN_3x_mkf_5_1024', 'COCO-Detection/retinanet_R_101_FPN_3x.yaml', 1024, False),
    ('retinanet_R_101_FPN_3x_mkf_5_1024_rm', 'COCO-Detection/retinanet_R_101_FPN_3x.yaml', 1024, False),
    ('retinanet_R_101_FPN_3x_mkf_5_512', 'COCO-Detection/retinanet_R_101_FPN_3x.yaml', 512, False),
    ('retinanet_R_50_FPN_3x_mkf_5_1024', 'COCO-Detection/retinanet_R_50_FPN_3x.yaml', 1024, False),
    ('retinanet_R_50_FPN_3x_mkf_5_512', 'COCO-Detection/retinanet_R_50_FPN_3x.yaml', 512, False),
    ('retinanet_R_50_FPN_3x_nms01_mkf_5_512', 'COCO-Detection/retinanet_R_50_FPN_3x.yaml', 512, False),
]

In [10]:
def inference(dataset_dicts, batch_size, outdir, predictor, metadata):
    results_list = []
    index = 0

    for i in tqdm(range(ceil(len(dataset_dicts) / batch_size))):
        inds = list(range(batch_size * i, min(batch_size * (i + 1), len(dataset_dicts))))
        dataset_dicts_batch = [dataset_dicts[i] for i in inds]
        im_list = [cv2.imread(d["file_name"]) for d in dataset_dicts_batch]
        outputs_list = predict_batch(predictor, im_list)

        for im, outputs, d in zip(im_list, outputs_list, dataset_dicts_batch):
            resized_height, resized_width, ch = im.shape
            outputs = predictor(im)
            pred_classes = outputs["instances"].to("cpu").get_fields()['pred_classes'].numpy()
            can_visualize = np.all((0 <= pred_classes) & (pred_classes <= len(classes_nms)))
            if index < 5 and can_visualize:
                # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
                v = Visualizer(
                    im[:, :, ::-1],
                    metadata=metadata,
                    scale=0.5,
                    instance_mode=ColorMode.IMAGE_BW
                    # remove the colors of unsegmented pixels. This option is only available for segmentation models
                )
                out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
                cv2.imwrite(str(outdir / f"pred_{index}.jpg"), out.get_image()[:, :, ::-1])

            image_id, dim0, dim1 = test_meta.iloc[index].values

            instances = outputs["instances"]
            if len(instances) == 0:
                # No finding, let's set 14 1 0 0 1 1x.
                result = {"image_id": image_id, "PredictionString": "14 1.0 0 0 1 1"}
            else:
                # Find some bbox...
                fields: Dict[str, Any] = instances.get_fields()
                pred_classes = fields["pred_classes"]  # (n_boxes,)
                pred_scores = fields["scores"]
                # shape (n_boxes, 4). (xmin, ymin, xmax, ymax)
                pred_boxes = fields["pred_boxes"].tensor

                h_ratio = dim0 / resized_height
                w_ratio = dim1 / resized_width
                pred_boxes[:, [0, 2]] *= w_ratio
                pred_boxes[:, [1, 3]] *= h_ratio

                pred_classes_array = pred_classes.cpu().numpy()
                pred_boxes_array = pred_boxes.cpu().numpy()
                pred_scores_array = pred_scores.cpu().numpy()

                mask = (0 <= pred_classes_array) & (pred_classes_array < len(classes_nms))  # RetinaNet's outputs uncorrectly contain unknown class_id

                result = {
                    "image_id": image_id,
                    "PredictionString": format_pred(
                        pred_classes_array[mask], pred_boxes_array[mask], pred_scores_array[mask]
                    ),
                }
            results_list.append(result)
            index += 1
    
    return pd.DataFrame(results_list, columns=['image_id', 'PredictionString'])

In [ ]:
torch.cuda.set_device(config.device_id)


for model_dir, model_config, img_size, change_anchor in tqdm(models):
    if model_dir not in ['faster_rcnn_R_101_FPN_3x_mkf_5_1024_rm', 'retinanet_R_101_FPN_3x_mkf_5_512']:
        continue
    print(model_dir)
    outdir = base_dir / config.outdir.replace('[tmp]', model_dir)
    dataset_nm = f'vinbigdata_test_{img_size}'
    if img_size == 512:
        dataset_dicts = dataset_dicts_512
        metadata = metadata_512
    elif img_size == 1024:
        dataset_dicts = dataset_dicts_1024
        metadata = metadata_1024
    else:
        raise ValueError()
        
    for fold in range(config.n_splits):
        print(f'fold: {fold}')
        seed_everything(seed=config.seed, device=config.device)

        cfg = get_cfg()
        original_output_dir = cfg.OUTPUT_DIR
        cfg.OUTPUT_DIR = str(outdir / f'fold-{fold + 1}')
        print(f"cfg.OUTPUT_DIR {original_output_dir} -> {cfg.OUTPUT_DIR}")

        cfg.merge_from_file(model_zoo.get_config_file(model_config))
        cfg.DATASETS.TRAIN = ("vinbigdata_train",)
        cfg.DATASETS.TEST = ()

        cfg.DATALOADER.NUM_WORKERS = config.num_workers
        # Let training initialize from model zoo
        cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(model_config)
        cfg.SOLVER.IMS_PER_BATCH = config.batch_size
        cfg.SOLVER.BASE_LR = config.base_lr  # pick a good LR
        cfg.SOLVER.MAX_ITER = config.iter
        cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = config.roi_batch_size_per_image
        cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(classes_nms)
        
        if change_anchor:
            cfg.MODEL.ANCHOR_GENERATOR.SIZES = [[16], [32], [64], [128], [256], [512], [1024]]
            cfg.MODEL.RPN.IN_FEATURES = ['p2', 'p2', 'p3', 'p4', 'p5', 'p6', 'p6']
            cfg.MODEL.ANCHOR_GENERATOR.ASPECT_RATIOS = [[0.33, 0.5, 1.0, 2.0, 3.0]]

        # NOTE: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.

        ### --- Inference & Evaluation ---
        # Inference should use the config with parameters that are used in training
        # cfg now already contains everything we've set previously. We changed it a little bit for inference:
        # path to the model we just trained
        cfg.MODEL.WEIGHTS = str(outdir / f'fold-{fold + 1}' / "model_final.pth")
        print("Original thresh", cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST)  # 0.05
        cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.0  # set a custom testing threshold
        print("Changed  thresh", cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST)
        predictor = DefaultPredictor(cfg)

        pred_df = inference(dataset_dicts=dataset_dicts, batch_size=config.batch_size, outdir=outdir / f'fold-{fold + 1}', predictor=predictor, metadata=metadata)
        pred_df.to_csv(str(outdir / f'fold-{fold + 1}' / 'submmission.csv'), index=False)

  0%|          | 0/9 [00:00<?, ?it/s]

faster_rcnn_R_101_FPN_3x_mkf_5_1024_rm
fold: 0
cfg.OUTPUT_DIR ./output -> /home/yamaguchi-milkcocholate/VinBigData/src/VinBigData-ObjectDetection/detectron2_results/faster_rcnn_R_101_FPN_3x_mkf_5_1024_rm/fold-1
Original thresh 0.05
Changed  thresh 0.0


  0%|          | 0/120 [00:00<?, ?it/s]